# Closures: Anonymous Functions that Can Capture Their Environment


Rust’s closures are anonymous functions you can save in a variable or pass as arguments to other functions. You can create the closure in one place and then call the closure to evaluate it in a different context.

The syntax and capabilities of closures make them very convenient for on the fly usage. Calling a closure is exactly like calling a function. However, both input and return types can be inferred and input variable names must be specified.

Other characteristics of closures include:

*   using `| |` instead of `()` around input variables.
*   optional body delimination (`{}`) for a single expression (mandatory otherwise).
*   the ability to capture the outer environment variables.


In [ ]:
fn main() {
    // Increment via closures and functions.
    fn  function (i: i32) -> i32 { i + 1 }

    // Closures are anonymous, here we are binding them to references
    // Annotation is identical to function annotation but is optional
    // as are the `{}` wrapping the body. These nameless functions
    // are assigned to appropriately named variables.
    let closure_annotated = |i: i32| -> i32 { i + 1 };
    let closure_inferred  = |i     |          i + 1  ;

    let i = 1;
    println!("function: {}", function(i));
    println!("closure_annotated: {}", closure_annotated(i));
    println!("closure_inferred: {}", closure_inferred(i));
}
main();

Unlike functions, closures can capture values from the scope in which they’re defined.

In [ ]:
fn main() {
    let value = 3;

    let f = |x| x + value; // the closure here refers to value

    println!("{}", f(2));
}
main();

By default, closures capture values as (immutable) references :

In [ ]:
fn main() {
    let value = String::from("Hello !");
    let tell = || { println!("{}", value); };
    tell();
    println!("value is still here: {}", value);
}
main();

Closures can also capture mutable references, by marking the variable that owns the closure as mutable :

In [ ]:
fn main() {
    let mut value = 0;
    {
        let mut inc = || { value += 1; };
        inc();
    }
    println!("value = {}", value);
}
main();

The borrow-checker ensures the safety of the mutable references even for closures. In the code above, remove the inner block to put all statements into the block of the `main` function and see what happens.

Sometimes it is useful for the closure to take ownership of its environment. For this purpose, use the `move` keyword in the closure definition. This will mean that the closure takes ownership and moves all the data it refers to :

In [ ]:
fn main() {
    let a = String::from("Hello ");
    let b = String::from("World !");
    let tell = move || { print!("{}", a); }; // here a is moved to the closure

    tell();
    println!("{}", b); // this is OK
    // println!("{}", a); // this is an error
}
main();

Closures don’t require you to annotate the types of the parameters or the return value like fn functions do. Closure definitions will have one concrete type inferred for each of their parameters and for their return value. 

Capturing the environment with closures
---

Closures can capture values from their environment in three ways, which directly map to the three ways a function can take a parameter: taking ownership, borrowing mutably, and borrowing immutably. These are encoded in the three `Fn` traits as follows:

* `FnOnce` consumes the variables it captures from its enclosing scope, known as the *closure’s environment*. To consume the captured variables, the closure must take ownership of these variables and move them into the closure when it is defined. The Once part of the name represents the fact that the closure can’t take ownership of the same variables more than once, so it can be called only once.
* `FnMut` can change the environment because it mutably borrows values.
* `Fn` borrows values from the environment immutably.


# Exercises

**Exercise 1**: Complete the following code

In [ ]:
fn adder(value: i32) -> impl Fn(i32) -> i32 {
    todo!("fixme");
}

fn test_adder() {
    assert_eq!(adder(3)(2), 5);
    assert_eq!(adder(4)(6), 10);
}
test_adder();

In [ ]:
fn closure_user<F: Fn(i32)->i32>(closure: F) {
    assert_eq!(closure(3), 4);
    assert_eq!(closure(5), 6);
    assert_eq!(closure(7), 8);
}

fn test_closure_user() {
    let mut call_count = 0;
    closure_user(|x| {
        call_count += 1;
        x + 1 
    });
    assert_eq!(call_count, 3);
}
test_closure_user();

**Solution**

In [ ]:
fn adder(value: i32) -> impl Fn(i32) -> i32 {
    move |x| x + value
}

fn test_adder() {
    assert_eq!(adder(3)(2), 5);
    assert_eq!(adder(4)(6), 10);
}
test_adder();


fn closure_user<F: FnMut(i32)->i32>(mut closure: F) {
    assert_eq!(closure(3), 4);
    assert_eq!(closure(5), 6);
    assert_eq!(closure(7), 8);
}

fn test_closure_user() {
    let mut call_count = 0;
    closure_user(|x| {
        call_count += 1;
        x + 1 
    });
    assert_eq!(call_count, 3);
}
test_closure_user();